In [6]:
# Package Imports
import dotenv
import openai

# Internal Imports
import os

### Prompt Design

One of the most important interactions between a user and OpenAI, is how we deisgn the prompt. A poorly designed prompt and handling can lead to inferior responses. A well designed prompt and handling can lead to better responses; responses that are more in line with what we are looking for. 

Additionally, poor prompt design and handling can lead to security issues. One of the usual methods of attack on any application is by manipulating user input. If we have a poorly designed prompt, we can be vulnerable to attacks. If we take the security of the prompt into question during design, we have less to worry about in the future.

#### Design Factors

* Model Choice
    * Unless there is a good reason not to, we should use the largest model available. This will give us the most options for the future. An example of a reason not to could be a zero day vulnerability in the model, or using an old model for specific testing purposes.

* Instructions
    * We can and should provide instructions to the model. If we want an SQL response, then design the prompt to ask for an SQL response. If we want a summary of text, ask for it. 
    * OpenAI has good guidelines on defining effective prompts and we should follow these. For example if we want SQL:
    
    ``` Translate the following text to an SQL prompt: ```
    
    ``` TEXT: """```
    
    ``` {Whatever text we can converted injected here} ```
    
    ``` """ ```
* Details
    * The more succint and appropriate details we provide, the better the response will be. For example if we want a macabre poem, the following would be a badly detailed prompt:
        * Write a macabre poem

    * The following would be a better prompt:
        * Write a macbre poem in iambic pentameter that is sixteen lines long and includes the words "raven", "owl" and "caw".

* Be Direct
    * As of right now, OpenAI and the models we use do not have any emotions. Be direct and clear in what you want.

* Initate the response in the prompt
    * This can be helpful, especially if we are excepting code in our response. If we want a python function, start the function at the end of the prompt:

    ``` Write a function that iterates over a list and removes any duplicates. def removeDuplicates(list):   ```

Alas, my description and examples are not exhaustive, nor am I more of an expert on the subject matter than those who designed it. For more detailed information on designing prompts, you can read OpenAI's documentation [here](https://platform.openai.com/docs/guides/completion/introduction/prompt-design).

#### Designing The Exam - A Catch

Currently, there is no way to be 100% sure that the answers we get are completely accurate. While there is some way to better the accuracy, we will need to wait until these models can fact-check themselves before we can completely rely on them.

In [5]:
# API Key is Stored in a .env File, please see the .env.example file for more information
dotenv.load_dotenv()
openai.api_key = os.getenv("OPENAPI")

#### The Exam Prompt

Here is a list of requirements that we want from the prompt. 

* It needs to create a question on a specific topic. We will do this by passing an int parameter ``` topic ```
* It needs to create x questions. We will do this by passing a variable ``` n_questions ```
* It needs to provide n answers. We will do this by passing an int parameter ``` n_answers ```
* It needs to provide one answer that is correct.
* It needs to provide n-1 answers that are incorrect.

In [9]:
def create_exam_prompt(topic, n_questions, n_answers):
    prompt = f"Create a multiple choice exam on the topic of {topic}. The exam should have {n_questions} questions. " \
                 + f"Each question should have {n_answers} options. "\
                 + f"Each questions should have one correct answer and {n_answers-1} incorrect answers. "\
                 + f"Also include the correct answer for each question using the starting string 'Correct Answer: '."
    return prompt

Now that we have designed a function that creates our prompt, let's test it out to see how it looks.

In [10]:
create_exam_prompt("Deterministic finite automaton", 5, 4)

"Create a multiple choice exam on the topic of Deterministic finite automaton. The exam should have 5 questions. Each question should have 4 options. Each questions should have one correct answer and 3 incorrect answers. Also include the correct answer for each question using the starting string 'Correct Answer: '."

#### OpenAI Call

Now we can easily create prompts to pass to OpenAI.

For this call we will use the following parameters:

* Engine: Text-Davinci-003

* Prompt: create_exam_prompt("Deterministic finite automaton", 5, 4)

* Max_Tokens = 256

* Temperature = 0.7

In [11]:
response = openai.Completion.create(engine="text-davinci-003",
                                            prompt=create_exam_prompt("Deterministic finite automaton", 5, 4),
                                            max_tokens=256,
                                            temperature=0.7)
response["choices"][0]["text"]

'\n\nQ1: What is a Deterministic Finite Automaton (DFA)?\nA) A theoretical machine used for solving mathematical problems\nB) A mathematical model of computation used to recognize patterns within input\nC) A type of graph used to represent an algorithm\nD) A type of computer used to run programs\n\nCorrect Answer: B) A mathematical model of computation used to recognize patterns within input\n\nQ2: What is the input to a DFA?\nA) An expression\nB) A set of instructions\nC) A set of symbols\nD) A set of numbers\n\nCorrect Answer: C) A set of symbols\n\nQ3: What is the output of a DFA?\nA) A set of instructions\nB) A set of symbols\nC) A set of numbers\nD) A yes/no answer\n\nCorrect Answer: D) A yes/no answer\n\nQ4: What is the purpose of a DFA?\nA) To solve mathematical problems\nB) To recognize patterns within input\nC) To represent an algorithm\nD) To run programs\n\nCorrect Answer: B) To recognize patterns within input\n\nQ5: How many states can a DFA'

As the above is hard to read, let's reformat it to make it more readable. First we can remove all of the '\n' characters, instead putting a new line there. Also, we will remove the 5th question as it got cut off, as the amount of tokens was too small.

Q1: What is a Deterministic Finite Automaton (DFA)?  
    A) A theoretical machine used for solving mathematical problems  
    B) A mathematical model of computation used to recognize patterns within input  
    C) A type of graph used to represent an algorithm  
    D) A type of computer used to run programs      
    Correct Answer: B) A mathematical model of computation used to recognize patterns within input      
    
Q2: What is the input to a DFA?  
    A) An expression  
    B) A set of instructions  
    C) A set of symbols  
    D) A set of numbers    
    Correct Answer: C) A set of symbols    

Q3: What is the output of a DFA?  
    A) A set of instructions  
    B) A set of symbols  
    C) A set of numbers  
    D) A yes/no answer    
    Correct Answer: D) A yes/no answer    

Q4: What is the purpose of a DFA?  
    A) To solve mathematical problems  
    B) To recognize patterns within input  
    C) To represent an algorithm  
    D) To run programs    
    Correct Answer: 
    B) To recognize patterns within input    

While it was simple for us to reformat this, the format what we choose would not be the intended use for this application. Instead, we want to create a viewpoint without the correct answer visible, but retain the knowledge of the correct answer for grading. We can do this by designing another function to extract the question, answers and correct answer for each question.

In [15]:
def create_exam_view(test, n_questions):
    exam_view = {1 : ""}
    question_n = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            exam_view[question_n] += line+"\n"
        else:

            if question_n < n_questions:
                question_n+=1
                exam_view[question_n] = ""
    return exam_view

In [20]:
def extract_correct_answers(test, n_questions):
    answers = {1 : ""}
    question_n = 1
    for line in test.split("\n"):
        if line.startswith("Correct Answer:"):
            answers[question_n] += line+"\n"

            if question_n < n_questions:
                question_n+=1
                answers[question_n] = ""
    return answers

In [17]:
create_exam_view(response["choices"][0]["text"], 4)

{1: '\n\nQ1: What is a Deterministic Finite Automaton (DFA)?\nA) A theoretical machine used for solving mathematical problems\nB) A mathematical model of computation used to recognize patterns within input\nC) A type of graph used to represent an algorithm\nD) A type of computer used to run programs\n\n',
 2: '\nQ2: What is the input to a DFA?\nA) An expression\nB) A set of instructions\nC) A set of symbols\nD) A set of numbers\n\n',
 3: '\nQ3: What is the output of a DFA?\nA) A set of instructions\nB) A set of symbols\nC) A set of numbers\nD) A yes/no answer\n\n',
 4: '\nQ4: What is the purpose of a DFA?\nA) To solve mathematical problems\nB) To recognize patterns within input\nC) To represent an algorithm\nD) To run programs\n\n\nQ5: How many states can a DFA\n'}

In [21]:
extract_correct_answers(response["choices"][0]["text"], 4)

{1: 'Correct Answer: B) A mathematical model of computation used to recognize patterns within input\n',
 2: 'Correct Answer: C) A set of symbols\n',
 3: 'Correct Answer: D) A yes/no answer\n',
 4: 'Correct Answer: B) To recognize patterns within input\n'}

#### Lets Take The Exam

Based on the above, we can now simulate an exam and see how well we do.

In [22]:
def take(exam_view):
    answers = {}
    for question, question_view in exam_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        answers[question] = answer
    return answers


In [35]:
def grade(correct_answers_list, answers):
    correct_answers = 0
    for question, answer in answers.items():
        if answer.upper() == correct_answers_list[question].upper()[16]:
            correct_answers+=1
    grade = 100 * correct_answers / len(answers)

    grammar = 'an'
    if grade >= 90:
        letter = 'A'
        grammar = 'an'
    elif grade >= 80:
        letter = 'B'
    elif grade >= 70:
        letter = 'C'
    elif grade >= 60:
        letter = 'D'
    else:
        letter = "E"
        grammar = 'an'
    return f"Exam results: {correct_answers} out of {len(answers)} answers correct! You achieved {grammar} {letter} with a percentage score of {grade}."


In [26]:
answers = take(create_exam_view(response["choices"][0]["text"], 4))



Q1: What is a Deterministic Finite Automaton (DFA)?
A) A theoretical machine used for solving mathematical problems
B) A mathematical model of computation used to recognize patterns within input
C) A type of graph used to represent an algorithm
D) A type of computer used to run programs



Q2: What is the input to a DFA?
A) An expression
B) A set of instructions
C) A set of symbols
D) A set of numbers



Q3: What is the output of a DFA?
A) A set of instructions
B) A set of symbols
C) A set of numbers
D) A yes/no answer



Q4: What is the purpose of a DFA?
A) To solve mathematical problems
B) To recognize patterns within input
C) To represent an algorithm
D) To run programs


Q5: How many states can a DFA



In [36]:
grade(extract_correct_answers(response["choices"][0]["text"], 4), answers)

'Exam results: 4 out of 4 answers correct! You achieved an A with a percentage score of 100.0.'